# 20 Questions

The intuition behind the algorithm is simple:
1. View WordNet as a collection of trees. Words are nodes. Relations are edges.
2. To formulate a question: Find the largest subtree. Query that relation.
3. To process the answer: If 'yes', enter the subtree. If 'no', backtrack to 2.

In [1]:
import nltk
from nltk.corpus import wordnet as wn

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zivlabstudent\AppData\Roaming\nltk_data...


True

In [4]:
wn.synset('waffle.n.01').definition()

'pancake batter baked in a waffle iron'

In [17]:
class Node:
    def __init__(self, label: str, definition: str, children=()):
        self.label = label
        self.definition = definition
        self.children = children

    def name(self):
        return self.label.rsplit('.')[0].replace('_', ' ')

    def defn(self):
        return self.definition
    
    def count(self):
        # vacuously returns 1
        return 1 + sum([c.count() for c in self.children])
    
    def find(self, target, path):
        if self.label == target:
            return path + [self.label]
        elif len(self.children) == 0:
            return []
        else:
            for child in self.children:
                c = child.find(target, path + [self.label])
                if len(c) > 0:
                    return c
            return []
            

In [18]:
# Only run on nouns!
def build_tree(root):
    hyponyms = root.hyponyms()
    if len(hyponyms) == 0:
        return Node(root.name(), root.definition())
    else:
        return Node(root.name(), root.definition(), [build_tree(child) for child in hyponyms])

In [113]:
POS = ('yes', 'y')
NEG = ('no', 'n')

def play_game(root=wn.synset('entity.n.01')):   
    tree = build_tree(root)
    subtree_ind = 0
    
    correct = False
    q_num = 0
    
    while not correct:
        subtrees = sorted(tree.children, key=Node.count, reverse=True)

        if len(subtrees) == 0:
            print(f'I win in {q_num} guesses! Your word is: {tree.name()}')
            return
        
        if subtree_ind >= len(subtrees):
            print('I could not guess your word :(')
            return

        current_topic = subtrees[subtree_ind]
        
        q_num += 1
        response = input(f'Question {q_num}: Is it a {current_topic.name()}?')

        if response.endswith('!'):
            print(f'I win in {q_num} guesses! Your word is "{current_topic.name()}"!')
        if response in POS:
            tree = current_topic
            subtree_ind = 0
        elif response in NEG:
            subtree_ind += 1
        else:
            print(f'\tA "{current_topic.name()}" is "{current_topic.definition}"')
            q_num -= 1

In [114]:
play_game(wn.synset('domestic_cat.n.01'))

Question 1: Is it a tom? n
Question 2: Is it a siamese cat? ?


	A "siamese cat" is "a slender short-haired blue-eyed breed of cat having a pale coat with dark ears paws face and tail tip"


Question 2: Is it a siamese cat? n
Question 3: Is it a manx? ?


	A "manx" is "a short-haired tailless breed of cat believed to originate on the Isle of Man"


Question 3: Is it a manx? y


I win in 3 guesses! Your word is: manx


In [20]:
t = build_tree(wn.synset('entity.n.01'))

In [22]:
t.find('waffle.n.01', [])

['entity.n.01',
 'physical_entity.n.01',
 'matter.n.03',
 'solid.n.01',
 'food.n.02',
 'baked_goods.n.01',
 'cake.n.03',
 'waffle.n.01']